In [9]:
!pip install pandas scikit-learn streamlit > /dev/null
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

print("✅ Libraries ready!")

✅ Libraries ready!


The system cannot find the path specified.


In [10]:
df = pd.read_csv("BML(in).csv")

print("✅ Loaded data shape:", df.shape)
print("✅ Columns:", df.columns.tolist())

# Define features and targets
features = ['Gender', 'Age', 'Height', 'Weight', 'BMI', 'Diabetes Pedigree', 'Glucose', 'Alcohol']
targets = ['Obesity Output', 'Cholesterol Output', 'GERD Output', 'Diabetes Output', 'nutritional Deficiency Ouput']

X = df[features]
y = df[targets]

# Handle missing values
X.fillna(X.mean(), inplace=True)
y.fillna(0, inplace=True)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train individual models for each target
models = {}
accuracies = {}

for target in targets:
    print(f"\n🔍 Training model for: {target}")
    y_target = y[target]

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_target, test_size=0.2, random_state=42)

    # Train model with optimized parameters
    model = RandomForestClassifier(
        n_estimators=150,
        max_depth=12,
        min_samples_split=6,
        class_weight='balanced',
        random_state=42
    )
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies[target] = acc

    # Store model
    models[target] = model

    print(f"✅ Accuracy: {acc:.2%}")

# Save models and scaler
with open("health_models.pkl", "wb") as f:
    pickle.dump(models, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Print final accuracies
print("\n📊 Final Model Performance:")
for target, acc in accuracies.items():
    print(f"{target.replace(' Output',''):<25}: {acc:.2%}")

print("\n🚀 All models trained and saved successfully!")

C:\Users\nomst\AppData\Local\Temp\ipykernel_15444\1041205947.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)
C:\Users\nomst\AppData\Local\Temp\ipykernel_15444\1041205947.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


✅ Loaded data shape: (3767, 14)
✅ Columns: ['Gender', 'Height', 'Age', 'Weight', 'BMI', 'Smoker', 'Diabetes Pedigree', 'Glucose', 'Alcohol', 'Obesity Output', 'Cholesterol Output', 'GERD Output', 'Diabetes Output', 'nutritional Deficiency Ouput']

🔍 Training model for: Obesity Output
✅ Accuracy: 100.00%

🔍 Training model for: Cholesterol Output
✅ Accuracy: 89.39%

🔍 Training model for: GERD Output
✅ Accuracy: 85.41%

🔍 Training model for: Diabetes Output
✅ Accuracy: 94.69%

🔍 Training model for: nutritional Deficiency Ouput
✅ Accuracy: 100.00%

📊 Final Model Performance:
Obesity                  : 100.00%
Cholesterol              : 89.39%
GERD                     : 85.41%
Diabetes                 : 94.69%
nutritional Deficiency Ouput: 100.00%

🚀 All models trained and saved successfully!


In [11]:
%%writefile app.py
import streamlit as st
import numpy as np
import pickle

# Load Models and Scaler
with open("health_models.pkl", "rb") as f:
    models = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# Enhanced Diet Plans
DIET_PLANS = {
    "Obesity Output": {
        "icon": "⚖️",
        "title": "Weight Management Plan",
        "recommendations": [
            "High-protein meals (chicken, fish, tofu)",
            "Low-carb vegetables (spinach, broccoli)",
            "Avoid processed and fried foods",
            "Drink 2L+ water daily",
            "Portion control techniques",
            "Healthy fats (avocados, nuts)"
        ],
        "action": "150+ mins exercise weekly"
    },
    "Cholesterol Output": {
        "icon": "❤️",
        "title": "Heart Health Plan",
        "recommendations": [
            "Daily oats and nuts",
            "Avoid red meat",
            "Leafy greens (kale, spinach)",
            "Olive oil instead of butter",
            "Fatty fish 2-3x/week",
            "Soluble fiber (beans, lentils)"
        ],
        "action": "Regular heart checkups"
    },
    "GERD Output": {
        "icon": "🔥",
        "title": "Acid Reflux Plan",
        "recommendations": [
            "Banana, oatmeal, ginger tea",
            "Avoid spicy/citrus foods",
            "Smaller, frequent meals",
            "No eating 3hrs before bed",
            "Elevate bed head 6-8 inches",
            "Identify trigger foods"
        ],
        "action": "Sleep on left side"
    },
    "Diabetes Output": {
        "icon": "🩸",
        "title": "Blood Sugar Control",
        "recommendations": [
            "Low GI foods (quinoa, sweet potato)",
            "Whole grains only",
            "No sugary drinks",
            "High fiber intake",
            "Balanced macros each meal",
            "Consistent meal timing"
        ],
        "action": "Monitor glucose regularly"
    },
    "Nurtitio0l Deficiency Ouput": {
        "icon": "🍎",
        "title": "Nutrition Boost Plan",
        "recommendations": [
            "Iron-rich foods (spinach, lentils)",
            "Vitamin D supplements",
            "B12 sources (eggs, dairy)",
            "Colorful fruits/veggies",
            "Calcium-rich foods",
            "Consider multivitamin"
        ],
        "action": "Annual blood tests"
    }
}

# Streamlit UI
st.set_page_config(layout="wide")
st.title("🏥 Personalized Health Risk Analyzer")

# Input Section
with st.container():
    st.header("📝 Your Health Profile")
    col1, col2 = st.columns(2)

    with col1:
        gender = st.radio("Gender", ["Male", "Female"], horizontal=True)
        age = st.slider("Age", 5, 100, 25)
        height = st.number_input("Height (cm)", 100, 250, 170)
        weight = st.number_input("Weight (kg)", 30, 200, 70)

    with col2:
        bmi = st.number_input("BMI", 10.0, 50.0, 22.0, 0.1)
        glucose = st.slider("Glucose Level (mg/dL)", 70, 300, 95)
        pedigree = st.slider("Diabetes Pedigree", 0.0, 2.5, 0.5, 0.01)
        alcohol = st.slider("Alcohol (units/week)", 0, 50, 0)

# Prediction
if st.button("🔍 Analyze My Health Risks", type="primary"):
    # Prepare input
    input_data = np.array([[
        1 if gender == "Male" else 0,
        age, height, weight, bmi,
        pedigree, glucose, alcohol
    ]])
    input_scaled = scaler.transform(input_data)

    # Get predictions
    results = {}
    for target, model in models.items():
        pred = model.predict(input_scaled)[0]
        proba = model.predict_proba(input_scaled)[0][1] if hasattr(model, "predict_proba") else None
        results[target] = {"prediction": pred, "probability": proba}

    # Display results
    st.markdown("---")
    st.header("📊 Your Health Report")

    any_risk = False
    for target, result in results.items():
        disease_name = target.replace(" Output", "")

        if result["prediction"] == 1:
            any_risk = True
            prob = result["probability"] * 100 if result["probability"] else 100
            plan = DIET_PLANS[target]

            with st.expander(f"{plan['icon']} {disease_name} Risk ({prob:.1f}%)", expanded=True):
                st.subheader(f"{plan['title']}")

                st.markdown("**Recommended Actions:**")
                for item in plan["recommendations"]:
                    st.markdown(f"- {item}")

                st.markdown(f"\n**Pro Tip:** {plan['action']}")

    if not any_risk:
        st.success("🎉 No significant health risks detected!")
        st.markdown("""
        **Maintain your health with:**
        - Balanced diet
        - Regular exercise
        - Good sleep
        - Stress management
        """)

st.markdown("---")
st.caption("Note: This tool provides general health information, not medical advice.")

Overwriting app.py
